In [1]:
import pandas as pd
import numpy as np
import nimfa
import pickle

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
#2020-2021S1
f = open("input_2020-01.pkl",'rb')
df_20S1 = pickle.load(f)

for i in range(2,4):
    f = open("input_2020-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S1 = df_20S1.add(df_add, fill_value= 0)
for i in range(1,4):
    f = open("input_2021-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S1 = df_20S1.add(df_add, fill_value= 0)

In [4]:
id_s = np.load('sample_user_id.npy', allow_pickle=True)
id_s
#sample data
df_20S1= df_20S1[df_20S1.index.isin(id_s)]

In [5]:
#全部都是0的會出現NA
df_sp = df_20S1.div(df_20S1.sum(axis=1), axis= 0, fill_value= None)
df_sp= df_sp.round(8)
df_sp= df_sp.fillna(0.0)

In [6]:
from sklearn.decomposition import NMF
K = 5
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random', l1_ratio = 1) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_sp)
H = nmf.components_    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [7]:
df_H = pd.DataFrame(H, columns=df_sp.columns)
df_H

,優惠活動/折扣/集點_amount,優酪乳/優格_低_amount,免洗餐具_低_amount,卡式爐/瓦斯罐_低_amount,堅果/穀物飲_低_amount,壽司/飯糰_低_amount,汽油_一般_amount,汽油_高_amount,洋芋類零食_一般_amount,海苔_一般_amount,...,投影機周邊_一般_amount,專業教科書/參考書_低_amount,裁紙機_一般_amount,其他燈具_低_amount,裁紙機_高_amount,握筆器_低_amount,液化石油氣_高_amount,其他思緒靈活_高_amount,其他思緒靈活_一般_amount,液化石油氣_一般_amount
0,0.073685,0.009623,0.000344,0.000245,0.029875,0.095975,0.000000,0.000000,0.022976,0.000894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.061377,0.012126,0.000364,0.000798,0.011827,0.097938,0.000000,0.023765,0.019093,0.005411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.252163,0.059382,0.000744,0.001247,0.148957,0.560262,0.000000,0.041481,0.071630,0.014908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.047654,0.012160,0.000257,0.000478,0.019182,0.063181,3.515519,0.399386,0.026885,0.007314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.044300,0.011261,0.000147,0.000722,0.015760,0.066272,0.000000,0.009796,0.021447,0.003332,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

,0
香菸_低_amount,3.195736e+00
香菸_一般_amount,4.126990e-01
現沖咖啡_低_amount,3.180162e-01
麵包/漢堡/蛋糕_低_amount,1.533836e-01
啤酒_一般_amount,1.188825e-01
讀卡機_高_amount,6.177481e-09
陀螺_低_amount,1.579329e-08
真空保鮮機周邊_低_amount,1.339225e-07
削筆器_低_amount,1.579513e-07
橡皮筋_高_amount,2.572090e-07


,1
店家名稱_低_amount,2.832459e+00
店家名稱_一般_amount,1.432112e+00
其他服飾配件_低_amount,1.973544e-01
無法分類_一般_amount,1.950028e-01
店家名稱_高_amount,1.640727e-01
大豆/植物異黃酮_低_amount,2.642173e-08
米酒_高_amount,9.898638e-08
電子防潮箱_低_amount,1.166176e-07
陀螺_低_amount,1.363751e-07
其他男性活力_低_amount,1.426103e-07


,2
其他料理_一般_amount,2.349459e+00
其他料理_低_amount,1.293406e+00
現沖咖啡_低_amount,1.188038e+00
麵包/漢堡/蛋糕_低_amount,8.850720e-01
壽司/飯糰_低_amount,5.602619e-01
桌球_低_amount,9.889152e-08
露營_低_amount,3.101798e-07
其他茶具用品_低_amount,3.792843e-07
電扇周邊_高_amount,4.534500e-07
其他電子_低_amount,6.023587e-07


,3
汽油_一般_amount,3.515519e+00
汽油_高_amount,3.993862e-01
汽油_低_amount,1.085132e-01
麵包/漢堡/蛋糕_低_amount,9.755727e-02
香菸_一般_amount,7.036040e-02
傢俱清潔/保養劑_低_amount,5.929723e-08
膳食纖維_低_amount,1.556019e-07
弦樂器_一般_amount,2.229472e-07
網球_高_amount,2.230399e-07
冰塊_高_amount,3.509456e-07


,4
無法分類_低_amount,2.793108e+00
無法分類_一般_amount,5.293866e-01
其他服飾配件_低_amount,9.903923e-02
麵包/漢堡/蛋糕_低_amount,9.333094e-02
電信/網路費_低_amount,7.209550e-02
眼鏡周邊_低_amount,4.160812e-08
其他文具圖書_低_amount,1.161282e-07
室內電話周邊_低_amount,1.610826e-07
電扇周邊_高_amount,2.006597e-07
陀螺_低_amount,2.710185e-07


In [9]:
#20-21S2
f = open("input_2020-04.pkl",'rb')
df_20S2 = pickle.load(f)

for i in range(5,7):
    f = open("input_2020-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S2 = df_20S2.add(df_add, fill_value= 0)
for i in range(4,7):
    f = open("input_2021-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S2 = df_20S2.add(df_add, fill_value= 0)

In [10]:
id_s = np.load('sample_user_id.npy', allow_pickle=True)
id_s
#sample data
df_20S2= df_20S2[df_20S2.index.isin(id_s)]

In [11]:
#全部都是0的會出現NA
df_sp = df_20S2.div(df_20S2.sum(axis=1), axis= 0, fill_value= None)
df_sp= df_sp.round(8)
df_sp= df_sp.fillna(0.0)

In [12]:
K = 5
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random', l1_ratio = 1) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_sp)
H = nmf.components_    

In [13]:
df_H = pd.DataFrame(H, columns=df_sp.columns)
df_H

,優惠活動/折扣/集點_amount,優酪乳/優格_低_amount,免洗餐具_低_amount,卡式爐/瓦斯罐_低_amount,堅果/穀物飲_低_amount,壽司/飯糰_低_amount,汽油_一般_amount,汽油_高_amount,洋芋類零食_一般_amount,海苔_一般_amount,...,投影機周邊_一般_amount,專業教科書/參考書_低_amount,裁紙機_一般_amount,其他燈具_低_amount,裁紙機_高_amount,握筆器_低_amount,液化石油氣_高_amount,其他思緒靈活_高_amount,其他思緒靈活_一般_amount,液化石油氣_一般_amount
0,0.020846,0.010214,0.000036,0.000529,0.020497,0.050354,3.091061,0.601476,0.021615,0.004949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.018723,0.000189,0.000353,0.010801,0.036590,0.000000,0.030778,0.020117,0.001860,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.121779,0.009534,0.001963,0.000465,0.020785,0.047813,0.000000,0.022745,0.025436,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.728202,0.098535,0.000526,0.001386,0.208743,0.524955,0.000000,0.055460,0.091378,0.012478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.007590,0.000303,0.000131,0.001944,0.039247,0.000000,0.003193,0.014224,0.002850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

,0
汽油_一般_amount,3.091061e+00
汽油_低_amount,6.529520e-01
汽油_高_amount,6.014763e-01
電費_低_amount,7.109941e-02
電信/網路費_低_amount,6.521592e-02
教學機_高_amount,1.065000e-07
福袋_低_amount,1.346452e-07
履歷表_一般_amount,1.709257e-07
變壓器_低_amount,1.761874e-07
電扇周邊_低_amount,1.791145e-07


,1
無法分類_低_amount,3.653945e+00
無法分類_一般_amount,4.819605e-01
電信/網路費_低_amount,1.134206e-01
會員費/訂閱費_一般_amount,8.961910e-02
其他服飾配件_低_amount,5.452726e-02
牛肉類罐頭_高_amount,7.245920e-08
冬瓜茶_高_amount,9.398718e-08
真空保鮮機_低_amount,2.192792e-07
保健箱/急救包_一般_amount,2.839186e-07
其他茶具用品_低_amount,3.804900e-07


,2
香菸_低_amount,3.318185e+00
香菸_一般_amount,2.754998e-01
啤酒_一般_amount,1.830675e-01
現沖咖啡_低_amount,1.390297e-01
優惠活動/折扣/集點_amount,1.217787e-01
雲端空間_低_amount,8.029974e-09
筍類罐頭_高_amount,2.893383e-08
打卡鐘/紙_低_amount,3.968073e-08
機車安全帽_低_amount,6.839595e-08
護貝機_低_amount,9.420601e-08


,3
現沖咖啡_低_amount,1.280555e+00
麵包/漢堡/蛋糕_低_amount,1.232703e+00
其他料理_低_amount,1.220601e+00
其他料理_一般_amount,1.204445e+00
優惠活動/折扣/集點_amount,7.282024e-01
繃帶_一般_amount,5.100218e-09
刺繡/縫紉用品_高_amount,6.583539e-08
弦樂器_一般_amount,7.579233e-08
教學機_高_amount,3.526379e-07
牛肉類罐頭_高_amount,5.571574e-07


,4
店家名稱_低_amount,2.343298e+00
店家名稱_一般_amount,1.219098e+00
其他服飾配件_低_amount,2.882415e-01
其他料理_一般_amount,1.981448e-01
店家名稱_高_amount,1.402609e-01
車用降溫用品_高_amount,6.199380e-08
教學機_高_amount,1.079069e-07
料理刷_高_amount,4.192276e-07
快煮壺/電熱水壺/瓶周邊_低_amount,4.287303e-07
快煮壺/電熱水壺/瓶周邊_高_amount,4.830273e-07


In [15]:
df_sp= df_sp.values
loss= df_sp- np.dot(W, H)
print(np.sum(loss))

2593.418099015549


In [16]:
#20-21S3
f = open("input_2020-07.pkl",'rb')
df_20S3 = pickle.load(f)

for i in range(8,10):
    f = open("input_2020-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S3 = df_20S3.add(df_add, fill_value= 0)
for i in range(7,10):
    f = open("input_2021-0"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S3 = df_20S3.add(df_add, fill_value= 0)
    
#sample data
df_20S3= df_20S3[df_20S3.index.isin(id_s)]

#全部都是0的會出現NA
df_sp = df_20S3.div(df_20S3.sum(axis=1), axis= 0, fill_value= None)
df_sp= df_sp.round(8)
df_sp= df_sp.fillna(0.0)

K = 5
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random', l1_ratio = 1) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_sp)
H = nmf.components_    

df_H = pd.DataFrame(H, columns=df_sp.columns)
df_H

for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

df_sp= df_sp.values
loss= df_sp- np.dot(W, H)
print(np.sum(loss))

,0
無法分類_低_amount,2.968901e+00
店家名稱_低_amount,1.455296e+00
其他料理_一般_amount,1.190411e+00
無法分類_一般_amount,9.264857e-01
其他料理_低_amount,7.584038e-01
變壓器_一般_amount,1.262021e-07
春聯/對聯_低_amount,2.260783e-07
動物油_低_amount,3.404374e-07
變壓器_低_amount,4.183666e-07
公益_高_amount,4.234666e-07


,1
優惠活動/折扣/集點_amount,2.958072e+00
現沖咖啡_低_amount,5.261984e-01
麵包/漢堡/蛋糕_低_amount,4.306366e-01
壽司/飯糰_低_amount,2.108263e-01
雞料理_低_amount,1.382009e-01
羊料理_低_amount,6.379154e-09
弦樂器_低_amount,4.220874e-08
毛巾架_低_amount,1.522701e-07
美髮剪_一般_amount,1.572250e-07
磨豆機_低_amount,2.586928e-07


,2
香菸_一般_amount,3.447155e+00
現沖咖啡_低_amount,7.091584e-01
麵包/漢堡/蛋糕_低_amount,2.839346e-01
蔬果汁/茶_低_amount,2.181573e-01
啤酒_一般_amount,2.053917e-01
手機外接相機/鏡頭_低_amount,4.670455e-08
奶精_低_amount,8.302663e-08
其他電源線_低_amount,1.636937e-07
皮鞋_高_amount,2.018534e-07
衣領/袖專用洗劑_低_amount,2.174771e-07


,3
汽油_高_amount,2.515430e+00
汽油_一般_amount,1.867168e+00
麵包/漢堡/蛋糕_低_amount,1.310515e-01
其他料理_一般_amount,1.086811e-01
電費_低_amount,8.083833e-02
香菸周邊_一般_amount,8.781454e-09
製麵包機_低_amount,4.773918e-08
藥盒_高_amount,1.142524e-07
旅遊書籍_一般_amount,1.600436e-07
其他茶具用品_高_amount,2.693591e-07


,4
香菸_低_amount,2.721805e+00
現沖咖啡_低_amount,1.830093e-01
啤酒_一般_amount,1.339707e-01
綠茶_低_amount,1.017984e-01
麵包/漢堡/蛋糕_低_amount,9.475804e-02
印表機/事務機周邊_低_amount,1.555604e-09
旅遊書籍_低_amount,1.245939e-07
車輪餅_一般_amount,1.798032e-07
GABA/色胺酸_低_amount,2.091578e-07
網球_高_amount,2.355563e-07


3243.6222357205584


In [18]:
#20-21S4
f = open("input_2020-10.pkl",'rb')
df_20S4 = pickle.load(f)

for i in range(11,13):
    f = open("input_2020-"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S4 = df_20S4.add(df_add, fill_value= 0)
for i in range(10,13):
    f = open("input_2021-"+str(i) +".pkl",'rb')
    df_add = pickle.load(f)
    df_20S4 = df_20S4.add(df_add, fill_value= 0)
    
#sample data
df_20S4= df_20S4[df_20S4.index.isin(id_s)]

#全部都是0的會出現NA
df_sp = df_20S4.div(df_20S4.sum(axis=1), axis= 0, fill_value= None)
df_sp= df_sp.round(8)
df_sp= df_sp.fillna(0.0)

K = 5
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random', l1_ratio = 1) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_sp)
H = nmf.components_    

df_H = pd.DataFrame(H, columns=df_sp.columns)
df_H

for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

df_sp= df_sp.values
loss= df_sp- np.dot(W, H)
print(np.sum(loss))

,0
汽油_一般_amount,2.618688e+00
汽油_高_amount,2.424543e+00
香菸_一般_amount,1.414101e-01
麵包/漢堡/蛋糕_低_amount,1.401118e-01
現沖咖啡_低_amount,1.231507e-01
其他茶具用品_高_amount,4.993150e-09
繃帶_一般_amount,8.965034e-08
室內電話周邊_低_amount,1.310405e-07
洗手機_低_amount,1.430682e-07
電扇周邊_低_amount,2.050821e-07


,1
無法分類_低_amount,3.046802e+00
無法分類_一般_amount,5.582929e-01
麵包/漢堡/蛋糕_低_amount,1.145770e-01
電信/網路費_低_amount,1.092365e-01
壽司/飯糰_低_amount,8.609348e-02
電子書_低_amount,3.606457e-08
教學機_低_amount,3.813951e-08
氣泡水機周邊_低_amount,7.522746e-08
行李秤/手提秤_一般_amount,7.650169e-08
其他電源線_低_amount,1.820331e-07


,2
其他料理_一般_amount,2.022668e+00
店家名稱_低_amount,1.796919e+00
其他料理_低_amount,1.367419e+00
店家名稱_一般_amount,1.311609e+00
麵包/漢堡/蛋糕_低_amount,7.161254e-01
其他海鮮罐頭_一般_amount,8.956621e-08
電視天線_低_amount,2.825466e-07
其他電源線_低_amount,2.978728e-07
翻譯機_低_amount,6.709669e-07
桌球_低_amount,9.321933e-07


,3
香菸_低_amount,3.301019e+00
香菸_一般_amount,5.716932e-01
現沖咖啡_低_amount,5.476356e-01
麵包/漢堡/蛋糕_低_amount,3.250759e-01
奶茶_低_amount,1.831839e-01
其他海鮮罐頭_一般_amount,6.286336e-08
皮蛋_高_amount,9.926952e-08
洗手機_一般_amount,1.384824e-07
春聯/對聯_低_amount,1.532051e-07
工具組/箱_一般_amount,1.643934e-07


,4
智慧型手機_高_amount,3.500209e+00
智能手錶/手環_高_amount,8.764439e-02
智慧型手機_一般_amount,3.385399e-02
平板電腦_一般_amount,3.351355e-02
無法分類_一般_amount,2.798672e-02
倒數計時器_一般_amount,2.437046e-09
玻璃清潔/保養劑_低_amount,1.824300e-08
米線/河粉_低_amount,4.689295e-08
電視電纜線_一般_amount,6.332949e-08
翻譯機_低_amount,1.583594e-07


2832.157237691528
